In [17]:
import pandas as pd
import pyodbc
import psycopg2
import os
from sqlalchemy import create_engine


connection_data = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server={ip};"
    "Database=AdventureWorks2022;"
    "UID={user};"  
    "PWD={password};" 
)


postgres_params = {
'host': '{ip}',
'database': 'AdventureWorksDW',
'user': '{user}',
'password': '{password}'
}

In [ ]:
sql_files = [
    'Dimensions\DIM\DimPromotion.sql',
    'Dimensions\DIM\DimSalesTerritory.sql',
    'Dimensions\DIM\DimCurrency.sql',
    'Dimensions\DIM\DimDepartmentGroup.sql',
    'Dimensions\DIM\DimGeography.sql',
    'Dimensions\DIM\DimProductCategory.sql',
    'Dimensions\DIM\DimProductSubcategory.sql',
    'Dimensions\DIM\DimProduct.sql',
    'Dimensions\DIM\DimCustomer.sql',
    'Dimensions\DIM\DimEmployee.sql',
    'Dimensions\DIM\DimReseller.sql',
#   'DimDate',
    'Dimensions\DIM\DimScenario.sql'
]

pg_table_names = [
    'public.dimpromotion',
    'public.dimsalesterritory',
    'public.dimcurrency',
    'public.dimdepartmentgroup',
    'public.dimgeography',
    'public.dimproductcategory',
    'public.dimproductsubcategory',
    'public.dimproduct',
    'public.dimcustomer',
    'public.dimemployee',
#   'public.dimdate',
    'public.dimreseller',
    'public.dimscenario',
]


def execute_sql_and_insert_to_pg(connection_data, postgres_params, sql_file, pg_table_name):
    try:
        
        sql_conn = pyodbc.connect(connection_data)
        sql_cursor = sql_conn.cursor()

        
        with open(sql_file, 'r') as file:
            sql_query = file.read()

        
        sql_cursor.execute(sql_query)

        
        rows = sql_cursor.fetchall()

        
        pg_conn = psycopg2.connect(**postgres_params)
        pg_cursor = pg_conn.cursor()

        
        for row in rows:
            
            row_data = tuple(row)

            pg_cursor.execute(f"INSERT INTO {pg_table_name} VALUES %s", (row_data,))

        
        pg_conn.commit()

        
        sql_cursor.close()
        sql_conn.close()
        pg_cursor.close()
        pg_conn.close()

        print(f'Dados do {sql_file} inseridos na tabela {pg_table_name} com sucesso.')

    except Exception as e:
        print(f'Ocorreu um erro:', str(e))

def load_dim_date(postgres_params):
    try:
        df_date = pd.read_csv('DimDate.csv')
        # conexao postgres
        pg_conn = psycopg2.connect(**postgres_params)
        pg_cursor = pg_conn.cursor()
        
        rows = [tuple(x) for x in df_date.to_numpy()]
        for row in rows:
            row_data = tuple(row)
            pg_cursor.execute(f"INSERT INTO public.dimdate VALUES %s", (row_data,))
        
        pg_conn.commit()
        pg_cursor.close()
        pg_conn.close()

        print(f'Dados do csv inserido com sucesso.')
    except Exception as e:
            print(f'Ocorreu um erro:', str(e))

for sql_file, pg_table_name in zip(sql_files, pg_table_names):
    execute_sql_and_insert_to_pg(connection_data, postgres_params, sql_file, pg_table_name)


load_dim_date(postgres_params)